<a href="https://colab.research.google.com/github/yuu-eguci/flower-stuff-lab/blob/main/yuueguci/inception_v3_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
# Google Drive をマウントします。
# NOTE: 左のトコをポチポチやってマウントすることも出来ますが、マウントすることを明示するほうが好みなのでしています。
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 17flowers dataset を取得します。
# NOTE: マイドライブに 17flowers.zip を置いてある必要があります。自分で用意してください。
#       /content は一時領域なので、数時間ごとに消滅します。
#       だからイチイチ、 unzip しています。
!unzip "/content/drive/MyDrive/datasets/17flowers.zip" -d "/content"

In [ ]:
# my_utils.convert_image_for_prediction を利用する準備を行います。
!git clone https://github.com/yuu-eguci/flower-stuff-lab.git

# my_utils を import するため、 flower-stuff-lab 内へ移動します。
import os
os.chdir('/content/flower-stuff-lab/yuueguci')

In [22]:
"""
この notebook に必要な import を行うセルです。
NOTE: 下の方のセルだけを実行することもあるため、 import は分離しておきます。
"""

# 学習済みモデル Inception-v3 を利用します。
# NOTE: 152層 CNN
#       ImageNet 学習済みモデル(なので画像は)
#       Google が作ったヤツ。
#       入力サイズは(299,299,3)
import keras.applications.inception_v3

# 他、必要なライブラリを import します。
# NOTE: 今回は試しに keras. から参照するようにしてみます。
import keras
import keras.preprocessing.image
import keras.models
import keras.layers
import keras.optimizers
import keras.callbacks

import numpy
import my_utils

# 各種定数もこのセルに定義します。
# データセットのパスです。
TRAIN_DIR = '/content/train_images'
TEST_DIR = '/content/test_images'
# 各 TRAIN_DIR に含まれる画像数です。
EACH_TRAIN_DIR_COUNT = 70
# 各 TEST_DIR に含まれる画像数です。
EACH_TEST_DIR_COUNT = 10
# n flowers です。
# NOTE: 元は 17flowers ですが、増やす予定です。
#       なので n flowers です。
CLASSES_FOR_N_FLOWERS = [
    'Bluebell',
    'Buttercup',
    'ColtsFoot',
    'Cowslip',
    'Crocus',
    'Daffodil',
    'Daisy',
    'Dandelion',
    'Fritillary',
    'Iris',
    'LilyValley',
    'Pansy',
    'Snowdrop',
    'Sunflower',
    'Tigerlily',
    'Tulip',
    'Windflower',
]
# Inception-v3 の入力サイズです。
INCEPTION_V3_TARGET_SIZE = (299, 299)

# 1st fitting を結果を保存する hdf5 です。
FIRST_FITTING_HDF5 = '/content/drive/MyDrive/hdf5/inception_v3_n_flowers_fine_tuning_1st_cell.hdf5'
FIRST_FITTING_LOG_DIR = '/content/drive/MyDrive/TensorBoardLogs/inception_v3_n_flowers_fine_tuning_1st_cell'
# 2nd fitting を結果を保存する hdf5 です。
SECOND_FITTING_HDF5 = '/content/drive/MyDrive/hdf5/inception_v3_n_flowers_fine_tuning_2nd_cell.hdf5'
SECOND_FITTING_LOG_DIR = '/content/drive/MyDrive/TensorBoardLogs/inception_v3_n_flowers_fine_tuning_2nd_cell'
# Prediction に使う画像です。
IMAGE_PATH = '/content/drive/MyDrive/images-for-prediction/for-n-flowers/sunflower.jpg'


In [ ]:
"""
データセットを用意するセルです。
"""

# Data Augmentation のための ImageDataGenerator を作ります。
image_data_generator_to_train = keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=10,
)
image_data_generator_to_test = keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255,
)

# ImageDataGenerator へ画像を与えます。
directory_iterator_for_training = image_data_generator_to_train.flow_from_directory(
    TRAIN_DIR,
    target_size=INCEPTION_V3_TARGET_SIZE,
    batch_size=16,
    class_mode='categorical',
    shuffle=True,
)
directory_iterator_for_test = image_data_generator_to_test.flow_from_directory(
    TEST_DIR,
    target_size=INCEPTION_V3_TARGET_SIZE,
    batch_size=16,
    class_mode='categorical',
    shuffle=True,
)

In [ ]:
"""
元の Inception-v3 を学習せず、新しい層のみ学習を行うセルです。
"""

# 学習済みモデルのインスタンスを取得します。
# NOTE: include_top=False により、これら↓の層が除かれます。
#       mixed10
#       avg_pool
#       flatten
#       predictions input:(None,2048) output:(None,1000)
base_model = keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False)

# "Global spatial average pooling layer" 追加。
# NOTE: GlobalAveragePooling2D により、
#       特徴マップ(batch_size,rows,cols,channels) -> (batch_size,channels)へ変換(圧縮)します。
#       これにより、 mixed10 の段階で(None,8,8,2048)だったデータが(None,2048)へ圧縮されます。
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
# 全結合層追加。
x = keras.layers.Dense(1024, activation='relu')(x)
# "Logistic layer"(物流層?)を追加。分類したいクラス数を指定します。
predictions = keras.layers.Dense(17, activation='softmax')(x)

# モデル完成です。
# NOTE: なんか、今んとこ base_model.output に層を追加していくことで
#       predictions ですでに model は完成しているってイメージだったから、こうやってくっつけないといけないんだー?
#       って思っています。
#       中身を理解していないと、変数名(predictions とか)にピンとこない。
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

# fit 前には compile が必要で、さらにその前に、新しい層のみ学習を行う設定を行います。
# NOTE: base_model を回すのか。 model の中身じゃないのか……よくわかっていないです。
# NOTE: 311層ありました。
for i, layer in enumerate(base_model.layers):
    layer.trainable = False

# fit 前には compile が必要です。
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
)

# fitting を行います。
model.fit(
    directory_iterator_for_training,
    steps_per_epoch=EACH_TRAIN_DIR_COUNT,
    epochs=5,
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=FIRST_FITTING_LOG_DIR),
        keras.callbacks.ModelCheckpoint(FIRST_FITTING_HDF5, save_best_only=True)
    ],
    verbose=1,
    validation_data=directory_iterator_for_test,
    validation_steps=EACH_TEST_DIR_COUNT,
)

# Evaluate します。
score = model.evaluate(directory_iterator_for_test, verbose=1)
print(dict(score=score))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/TensorBoardLogs/inception_v3_n_flowers_fine_tuning_1st_cell

In [ ]:
"""
適当なところで層を区切って学習を行うセルです。
"""
TENSOR_BOARD_LOG_DIR = '/content/drive/MyDrive/TensorBoardLogs/inception_v3_n_flowers_fine_tuning_2nd_cell'
PREVIOUS_HDF5_FILE_PATH = '/content/drive/MyDrive/hdf5/inception_v3_n_flowers_fine_tuning_1st_cell.hdf5'
HDF5_FILE_PATH = '/content/drive/MyDrive/hdf5/inception_v3_n_flowers_fine_tuning_2nd_cell.hdf5'

# ひとつ前の fitting を行った model をロードします。
model = keras.models.load_model(PREVIOUS_HDF5_FILE_PATH)

# Fine-tuning を行います。250層目以降を trainable にします。
for i, layer in enumerate(model.layers[:249]):
    layer.trainable = False
print(i)
for i, layer in enumerate(model.layers[249:]):
    layer.trainable = True
print(i)

# fit 前には compile が必要です。
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9),
    loss='categorical_crossentropy',
)

# fitting を行います。
model.fit(
    directory_iterator_for_training,
    steps_per_epoch=EACH_TRAIN_DIR_COUNT,
    epochs=20,
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=TENSOR_BOARD_LOG_DIR),
        keras.callbacks.ModelCheckpoint(HDF5_FILE_PATH, save_best_only=True)
    ],
    verbose=1,
    validation_data=directory_iterator_for_test,
    validation_steps=EACH_TEST_DIR_COUNT,
)

# Evaluate します。
score = model.evaluate(directory_iterator_for_test, verbose=1)
print(dict(score=score))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/TensorBoardLogs/inception_v3_n_flowers_fine_tuning_2nd_cell

In [23]:
"""
Prediction を行うセルです。
NOTE: hdf5 を読み込んで predict します。
      毎回 fit -> predict なんてやってられないからです。
"""


def predict_top_class(model, img_nad):
    predicted = model.predict(img_nad, batch_size=1, verbose=1)
    predicted = numpy.argmax(predicted, axis=-1)
    return [CLASSES_FOR_N_FLOWERS[i] for i in predicted]


def predict_top5_classes(model, img_nad):
    prediction = model.predict(img_nad)[0]
    top_indices = prediction.argsort()[-5:][::-1]
    return [(CLASSES_FOR_N_FLOWERS[i], prediction[i]) for i in top_indices]


# 画像を読み込みます。
img_nad = my_utils.convert_image_for_prediction(IMAGE_PATH, INCEPTION_V3_TARGET_SIZE, 'rgb')

# 1st model で prediction してみます。
model = keras.models.load_model(FIRST_FITTING_HDF5)
print('1st model', predict_top_class(model, img_nad))
print('1st model', predict_top5_classes(model, img_nad))

# 2nd model で prediction してみます。
model = keras.models.load_model(SECOND_FITTING_HDF5)
print('1st model', predict_top_class(model, img_nad))
print('1st model', predict_top5_classes(model, img_nad))

load_img の返り値: <class 'PIL.Image.Image'>
PIL.Image.Image size: (299, 299)
img_to_array の返り値: <class 'numpy.ndarray'>
正規化後の img: <class 'numpy.ndarray'>
正規化後の img.shape: (299, 299, 3)
1/1 [==============================] - 8s 8s/step
1st model ['Sunflower']
1st model [('Sunflower', 0.999826), ('Windflower', 0.00013694806), ('ColtsFoot', 2.40964e-05), ('Tulip', 8.909642e-06), ('Tigerlily', 1.3229544e-06)]
1/1 [==============================] - 7s 7s/step
1st model ['Sunflower']
1st model [('Sunflower', 0.9999089), ('Daisy', 1.7981178e-05), ('Windflower', 1.5335736e-05), ('Snowdrop', 1.4135974e-05), ('Bluebell', 1.2670451e-05)]
